## Imports

In [1]:
# import NALU files

# import Keras items
from keras.models import Model
from keras.layers import Dense,Lambda,Concatenate,Input
from keras import backend as K
# from keras.utils import to_categorical,plot_model
from keras.initializers import constant

# import other libs
import numpy as np
# from sklearn.model_selection import train_test_split


Using TensorFlow backend.


## Parameters

In [33]:
model_name = 'tester'
EPOCHS = 100
BATCH_SIZE = 4
range_min = 0
range_max = 999
train_samples = 1500

## Generate Dataset

In [13]:
x = np.random.randint(low = 0, high = 10,size = (100,2))
y = x[:,0] * x[:,1]

## Model

In [4]:
def net(inputs):
    input_data = Input(shape = inputs)
    out = Dense(30,activation = 'relu',name='my_layer')(input_data)
    out = Dense(82,activation='softmax')(out)
    
    return Model(inputs = input_data,outputs = out)

+ #### Model Initilization And Details

In [5]:
model_mul = net((2,))
model_mul.trainable = False
model_mul.compile(loss = 'categorical_crossentropy',optimizer = 'Adam',metrics = ['accuracy'])
# model_mul.summary()
model_mul.load_weights('mul_0-9_classifier.h5')

In [19]:
def net_2(inputs):
    input_data = Input(shape=inputs)
    out = model_mul(input_data)
    out = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(out)
    return Model(inputs = input_data,outputs = out)    

In [20]:
model_2 = net_2((2,))
model_2.compile(loss = 'mse',optimizer = 'Adam',metrics = ['accuracy'])
model_2.trainable = False

In [ ]:
def net_mul(inputs):
    input_data = Input(shape = inputs)
    n = input_data[0].shape[-1]
    m = input_data[1].shape[-1]
    # get initial products
    products=[]
    for i in range(input_data.shape[-1]):
        mul = Lambda(lambda x: x[:,0,i:i+1])(input_data)
        for j in range(input_data.shape[-1]):
            a = Lambda(lambda x: x[:,1,j:j+1])(input_data)
            concat = Concatenate()([a,mul])
            prod = model_mul(concat)
            prod =  Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(prod)
            products.append(prod)
            
    # get individual rows
    products = products[::-1]
    row_values = []
    prod_index = 0
    for r in range(n):
        for i in range(m+1):
            if i==0:
                sep = model_separator(products[prod_index])
                ones_digit = Lambda(lambda x: x[:,1:])(sep)
                carry = Lambda(lambda x: x[:,0:1])(sep)
                row_values.append(ones_digit)
                prod_index+=1
                
            elif i == m :
                row_values.append(carry)
            else:
                concat = Concatenate()([products[prod_index],carry])
                addition =  Dense(1,use_bias=False,kernel_initializer='ones')(concat)
                sep = model_separator(addition)
                ones_digit = Lambda(lambda x: x[:,1:])(sep)
                carry = Lambda(lambda x: x[:,0:1])(sep)
                row_values.append(ones_digit)
                prod_index+=1
                
    final_row = []
    current_ind = 0
    for i in range(m+n):
        if i == 0:
            final_row.append(row_values[0])
            sep = model_separator(row_values[0])
            carry = Lambda(lambda x: x[:,0:1])(sep)
            current_ind+=1
        else:
            to_add = []
            to_add.append(row_values[current_ind])
            to_add.append(carry)
            next_ind = current_ind + m
            while(next_ind < n * (m+1) and next_ind%(m+1)!=0):
                to_add.append(row_values[next_ind])
                next_ind+=m
            if (next_ind < n * (m+1) and next_ind%(m+1)==0):
                to_add.append(row_values[next_ind])
                
            concat = Concatenate()(to_add)
            addition =  Dense(1,use_bias=False,kernel_initializer='ones')(concat)
            sep = model_separator(addition)
            ones_digit = Lambda(lambda x: x[:,1:])(sep)
            carry = Lambda(lambda x: x[:,0:1])(sep)
            final_row.append(ones_digit)
            
            if (current_ind + 1) % (m+1) !=0:
                current_ind+=1
            else:
                current_ind+=m+1

    
    concat = Concatenate()(final_row[::-1])
    return Model(inputs = input_data,outputs =concat)

+ #### Fit Model

+ #### Test Model

In [21]:
model_2.evaluate(x,y)

100/100 [==============================] - 0s 1ms/step


[0.0, 1.0]

In [55]:
out = model.predict(x)
for i in range(100):
    print ('\t',x[i],' = ', y[i])
    print('\t predicted: ',out[i])

	 0  =  0.0
	 predicted:  [ 0.]
	 1  =  0.0
	 predicted:  [ 0.]
	 2  =  0.0
	 predicted:  [ 0.]
	 3  =  0.0
	 predicted:  [ 0.]
	 4  =  0.0
	 predicted:  [ 0.]
	 5  =  0.0
	 predicted:  [ 0.]
	 6  =  0.0
	 predicted:  [ 0.]
	 7  =  0.0
	 predicted:  [ 0.]
	 8  =  0.0
	 predicted:  [ 0.]
	 9  =  0.0
	 predicted:  [ 0.]
	 10  =  1.0
	 predicted:  [ 1.]
	 11  =  1.0
	 predicted:  [ 1.]
	 12  =  1.0
	 predicted:  [ 1.]
	 13  =  1.0
	 predicted:  [ 1.]
	 14  =  1.0
	 predicted:  [ 1.]
	 15  =  1.0
	 predicted:  [ 1.]
	 16  =  1.0
	 predicted:  [ 1.]
	 17  =  1.0
	 predicted:  [ 1.]
	 18  =  1.0
	 predicted:  [ 1.]
	 19  =  1.0
	 predicted:  [ 1.]
	 20  =  2.0
	 predicted:  [ 2.]
	 21  =  2.0
	 predicted:  [ 2.]
	 22  =  2.0
	 predicted:  [ 2.]
	 23  =  2.0
	 predicted:  [ 2.]
	 24  =  2.0
	 predicted:  [ 2.]
	 25  =  2.0
	 predicted:  [ 2.]
	 26  =  2.0
	 predicted:  [ 2.]
	 27  =  2.0
	 predicted:  [ 2.]
	 28  =  2.0
	 predicted:  [ 2.]
	 29  =  2.0
	 predicted:  [ 2.]
	 30  =  3.0
	 predi

+ #### Save Model

In [14]:
model.save_weights('adder_0-9_classifier.h5')

In [151]:
plot_model(model,to_file = 'mul_0-9.png',show_shapes=True)